# Deploying end-to-end machine learning workflows with HPE Ezmeral ML Ops - Lab 3
## Model training

### **Lab workflow**

In this lab: 

1. You will first create your tenant user's working directory on the central project repository needed in the machine learning (ML) pipeline to store the dataset, trained models and scoring scripts.

2. You will then train and test the model on a dataset using a remote tenant-shared training cluster. 

#### About the Dataset
The dataset is based on the 2019 New York City yellow cab trip data (approximately 375,000 trip records from January-June 2019). The dataset has many different properties (aka _"X - features"_) like the pickup time and location, the dropoff time and location, the trip distance, the number of passengers, and several other variables. The goal is to predict the taxi ride duration in NY City (the target aka _"Y - label"_) based on these features. 

>**Note: _For this workshop, you will be using a premade subset of this dataset that requires little data preprocessing and preparation (i.e.: data cleaning, data transformation)._**

### **1- Code versioning integrated with Jupyter Notebook cluster**

As you can see in the left hand side, as a data scientist, you have all the files (for example notebooks and code scripts) you need to do your work. These files are all pulled from the GitHub source control repository set up by the Operations team for the data science team.

The Jupyter Notebook ML Ops application includes the **Git** plugin that allows data scientists to use Git and do version control of their notebooks and model code scripts directly from within their local Jupyter Notebook.

From within the local Jupyter Notebook, data scientists can do the usual _git status_, _git add_, _git commit_, _git push_ to push their notebooks to their GitHub repository branch from the _**Notebook terminal**_ provided, and start versioning their notebooks and codes, and collaborate across projects.
If there are other changes that are pushed by other data scientists in the same GitHub repository branch, user can pull up the changes from the terminal using _git pull_ command. 

<img src="Launcher-terminal.jpg" height="600" width="600">

### Learn more about Git

Want to learn more about Git, check out the community blog series and the HPE DEV workshop-on-demand:

* [Getting started with Git - blog series](https://developer.hpe.com/blog/get-involved-in-the-open-source-community-part-1-getting-started-with-git/)
* [Git 101 - Get involved in the open source community - Workshop-on-demand](https://hackshack.hpedev.io/workshops)

### **2- Setup your working directory**

The Python code here is used to create the working directory in the central project repository for your tenant userID. The working directory is used to store the key data components needed in the ML pipeline, such as the input dataset, trained ML model(s) and scoring script(s). 

In [ ]:
userID = "student{{ STDID }}"
smalldemodataset = True

import os
import shutil
import time

# Shared dataset across all tenant users, use case directory name and scoring file name
if smalldemodataset is True:
    datasetFile = "demodata-small.csv"
else:
    datasetFile = "demodata.csv"
    
locationTable = "lookup-ipyheader.csv"
usecaseDirectory = "NYCTaxi"
scoringFile="XGB_Scoring.py"
scoringFileV2="XGB_Scoringv2.py"

# Project repo path function - file system mount available to all app containers
def ProjectRepo(path):
    ProjectRepo = "/bd-fs-mnt/TenantShare/repo"
    return str(ProjectRepo + '/' + path)

print ("Creating your working directory...")

# Delete existing working directory for userID (that may exists from previous execution of the workshop)
studentRepoModel = "models" + '/' + usecaseDirectory + '/' + userID
studentRepoCode = "code" + '/' + usecaseDirectory + '/' + userID
pathModel = ProjectRepo(studentRepoModel)
pathCode = ProjectRepo(studentRepoCode)

if (os.path.exists(ProjectRepo(studentRepoModel))):
    #print ("Repository " + pathModel + " exists for user " + userID + ". Deleting the repo now...")
    shutil.rmtree(pathModel, ignore_errors=True)
        
if (os.path.exists(ProjectRepo(studentRepoCode))):
    #print ("Repository " + pathCode + " exists for user " + userID + ". Deleting the repo now...")
    shutil.rmtree(pathCode, ignore_errors=True)
    
# Making sure the input Dataset is loaded and accessible in the shared persistent container storage for your tenant
# Check the dataset file exists in /db-fs-mnt/TenantShare/repo/data/NYCTaxi folder:
#print (os.listdir(ProjectRepo('data/NYCTaxi')))
datasetFilePath = "data" + '/' + usecaseDirectory + '/' + datasetFile
locationFilePath = "data" + '/' + usecaseDirectory + '/' + locationTable
pathData = ProjectRepo("data" + '/' + usecaseDirectory)

if (not os.path.exists(ProjectRepo(datasetFilePath))):
    print ("Error! Dataset file " + ProjectRepo(datasetFilePath) + " does not exist. Please contact the workshop administrator at hpedev.hackshack@hpe.com before continuing.")
if (not os.path.exists(ProjectRepo(locationFilePath))):
    print ("Error! location table file " + ProjectRepo(locationFilePath) + " does not exist. Please contact the workshop administrator at hpedev.hackshack@hpe.com before continuing.")
    
print ("Demo dataset is " + datasetFilePath)

# Define the directory structure for the UserID to store trained model and the scoring script
# /bd-fs-mnt/TenantShare/repo/models/NYCTaxi/userID; /bd-fs-mnt/TenantShare/repo/code/NYCTaxi/userID

if (not os.path.exists(ProjectRepo(studentRepoModel))):
    print ("Creating the Model directory " + pathModel)
    try:
        os.makedirs(pathModel)
    except OSError:
        print ("Creation of the model directory %s failed" % pathModel)
    else:
        print ("Successfully created the model directory %s" % pathModel)

if (not os.path.exists(ProjectRepo(studentRepoCode))):
    print ("Creating the Code directory " + pathCode)
    try:
        os.makedirs(pathCode)
    except OSError:
        print ("Creation of the code directory %s failed" % pathCode)
    else:
        print ("Successfully created the code directory %s" % pathCode)
        
# Copying scoring script files to your code repository in your working directory
# Make sure the scoring script files are available in the /bd-fs-mnt/TenantShare/repo/code/NYCTaxi/userID folder with appropriate permissions (read-execute)
##print ("local directory on your local Jupyter Notebook:")
##print (os.listdir(os.curdir))
##print ("target directory on the shared File System mount for your tenant: ")
##print (os.listdir(pathCode))
srcFile = scoringFile
destFile = pathCode + '/' + scoringFile
##srcFileV2 = scoringFileV2
##destFileV2 = pathCode + '/' + scoringFileV2

if (not os.path.exists(scoringFile)):
    print ("")
    print ("Error! " + scoringFile + " does not exist in your local Jupyter Notebook! Please make sure to copy the file " + scoringFile + " to your local Jupyter Notebook, then re-run that code cell to continue the lab.")
else:    
    if (os.path.exists(pathCode + '/' + scoringFile)):
        print (pathCode + '/' + scoringFile + " file exists. Setting permissions")
        os.chmod (destFile, 0o777)
    else:
        #print ("File" + pathCode + '/' + scoringFile + " does not exist.")
        print ("Copying the scoring script on " + pathCode + " and setting file permissions")
        try:
            shutil.copy(srcFile, destFile)
            os.chmod (destFile, 0o777)
        except IOError as e:
            print ("Unable to copy scoring file. %s" % e )
        except:
            print ("Unexpected error:", sys.exec_info())

##if (not os.path.exists(scoringFileV2)):
##    print ("")
##    print ("Error! " + scoringFileV2 + " does not exist in your local Jupyter Notebook! Please make sure to copy the file " + scoringFileV2 + " to your local Jupyter Notebook, then re-run that code cell to continue the lab.")
##else:    
##    if (os.path.exists(pathCode + '/' + scoringFileV2)):
##        print (pathCode + '/' + scoringFileV2 + " file exists. Setting permissions")
##        os.chmod (destFileV2, 0o777)
##    else:
##        #print ("File" + pathCode + '/' + scoringFileV2 + " does not exist.")
##        print ("Copying the scoring script v2 on " + pathCode + " and setting file permissions")
##        try:
##            shutil.copy(srcFileV2, destFileV2)
##            os.chmod (destFileV2, 0o777)
##        except IOError as e:
##            print ("Unable to copy scoring file. %s" % e )
##        except:
##            print ("Unexpected error:", sys.exec_info())
            
#print ("target directory  " + pathCode + " content :")
#print (os.listdir(pathCode))


### **3- Test Connection to the remote tenant-shared training cluster**

Next, before training your model on remote shared training cluster, you may want to test that the communication with the shared training cluster is indeed functioning properly.

> **Note:** _The Jupyter Notebook ML Ops application used in our solution includes **custom magic** functions to handle remotely submitting training code and retrieving results and logs. The Notebook uses these magic functions to make REST API calls to the API server that runs as part of the shared training environment. These calls submit training jobs and get results from within the Notebook session._

The Jupyter Notebook ML Ops app includes the following **line magic** functions: 
*	%attachments: Returns a list of connected training environments. 
*	%logs --url: URL of the training server load balancer used to monitor the status of the training job.

The Jupyter Notebook ML Ops app also includes the following **cell magic** function:
*	%%training_cluster_name 
This would submit training code to the shared training environment

#### **Get the list of connected training environments**

<b>%attachments</b> is a line magic command that output a table with the name(s) of the training cluster(s) available for us to use. Sometimes, Operations team may have created multiple training clusters for different projects depending on the needs of the model or size of data, e.g. some with GPU nodes, while others with CPUs only.

In [ ]:
%attachments

#### **Submit a notebook code cell to a remote training cluster**

To utilize the training cluster, you will need grab the name of the training cluster you want to use and feed it into another custom cell magic command. 

With the **%%trainingengineshared** magic command specified at the beginning of the code cell, the Jupyter Notebook will 
submit the entire content of the cell to the training cluster named _trainingengineshared_. If you comment this magic command, the code will run on your local Jupyter Notebook.

The example cell below will execute a print statement on the training cluster.

In [ ]:
%%trainingengineshared

import datetime

print('test')
print("Date time: ", datetime.datetime.now())

#### **Retrieve the result of the job**

The training cluster will send back a unique log url for the job submitted.   
You can use the _History URL_ with the _"%log --url"_ custom **line magic** command to track the status of the job in real time. 

Copy the History URL from the output of the previous cell and paste it into the cell below where it says _"your_http_url_here"_, and run the cell code.

A status of "**Finished**" means the execution of the job submitted to the training cluster has completed.

In [ ]:
##%logs --url your_http_url_here
%logs --url your_http_url_here

### **4- Train the model on the remote shared training cluster**

In general, data scientists use their local Jupyter Notebook to **experiment** several learning algorithms with a variety of parameters. They do so to determine the ML model that works best for the business problem they try to address and develop the model that yields to the best prediction result. Then, within their notebooks, they submit their code to large scaled computing training cluster environment to train and test their full ML models, in a reasonable time, typically against a larger training dataset and test dataset. The output of this step is a trained model ready for deployment in production.

>**Note:** _This workshop is not intended to teach you about AI/ML model experimentation and development. It is intended to give a use case for data science end-to-end ML workflow with HPE Ezmeral ML Ops. Therefore we will assume that the experimentation step has already been done and that the data science team has shared the best performant ML model in a notebook in the GitHub version control system repository set up by the Operations team for the data science team. The notebook is actually this notebook pulled from GitHub repository by the local Jupyter Notebook cluster. Here you will submit the ML model code to the tenant-shared training cluster environment to train and test your model against the taxi ride training/test dataset._ 

#### About the machine learning model development workflow
Gradient boosting supervised machine learning algorithm with Python is used here to build the model that is capable of predicting the duration of taxi trips in New York city. Python libraries such as Numpy, Pandas, Scikit-learn, XGBoost are used to build the model. 

The machine learning workflow depicted in the diagram below follows the typical supervised machine learning workflow for models development:

<img src="ML-Workflow.jpg" height="600" width="600" align="right">

- After loading the dataset from the central project repository, the ML algorithm separates data into features (the taxi ride properties) and label (the taxi ride duration). 
- Then the dataset is divided into two parts, one for training the model and one for testing the model. The typical split is 80/20.
- The ML algorithm is then defined and the model is built with the training data set to learn from. 
- The resulting model is then run on the test data which was not used to train the model.
- Next, the model accuracy is evaluated by comparing the test predictions to the test labels. Error metrics such as RMSE are used here to evaluate the predictions accuracy.
- The trained model is finally saved to a file in the central project repository. The model is ready for deployment in production to serve predictions.
- The next step is then to move your trained model to production to serve your model as a prediction service (Lab Part 4) and make predictions on new data points (Lab Part 5).

In [ ]:
%%trainingengineshared

userID = "student{{ STDID }}"
smalldemodataset = True

print("Importing libraries")
import pandas as pd
import numpy as np
from scipy import stats
import math
import os
import datetime
import xgboost as xgb
import pickle

import matplotlib.pyplot as plt

usecaseDirectory = "NYCTaxi"

if smalldemodataset is True:
    datasetFile = "demodata-small.csv"
else:
    datasetFile = "demodata.csv"
    
locationTable = "lookup-ipyheader.csv"
datasetFilePath = "data" + '/' + usecaseDirectory + '/' + datasetFile
locationFilePath = "data" + '/' + usecaseDirectory + '/' + locationTable
studentRepoModel = "models" + '/' + usecaseDirectory + '/' + userID

# Start time 
print("Start time for " + userID + ": ", datetime.datetime.now())

# Project repo path function
def ProjectRepo(path):
    ProjectRepo = "/bd-fs-mnt/TenantShare/repo"
    return str(ProjectRepo + '/' + path)


print("Reading in data")
# Reading in dataset table using pandas
dbName = "pqyellowtaxi"
##df = pd.read_csv(ProjectRepo('data/NYCTaxi/demodata.csv'))
df = pd.read_csv(ProjectRepo(datasetFilePath))

# Reading in latitude/longitude coordinate lookup table using pandas 
lookupDbName = "pqlookup"
##dflook = pd.read_csv(ProjectRepo('data/NYCTaxi/lookup-ipyheader.csv'))
dflook = pd.read_csv(ProjectRepo(locationFilePath))
print("Done reading in data")


# merging dataset and lookup tables on latitudes/coordinates
df = pd.merge(df, dflook[[lookupDbName + '.location_i', lookupDbName + '.long', lookupDbName + '.lat']], how='left', left_on=dbName + '.pulocationid', right_on=lookupDbName + '.location_i')
df.rename(columns = {(lookupDbName + '.long'):(dbName + '.startstationlongitude')}, inplace = True)
df.rename(columns = {(lookupDbName + '.lat'):(dbName + '.startstationlatitude')}, inplace = True)
df = pd.merge(df, dflook[[lookupDbName + '.location_i', lookupDbName + '.long', lookupDbName + '.lat']], how='left', left_on=dbName + '.dolocationid', right_on=lookupDbName + '.location_i')
df.rename(columns = {(lookupDbName + '.long'):(dbName + '.endstationlongitude')}, inplace = True)
df.rename(columns = {(lookupDbName + '.lat'):(dbName + '.endstationlatitude')}, inplace = True)


def fullName(colName):
    return dbName + '.' + colName

# convert string to datetime
df[fullName('tpep_pickup_datetime')] = pd.to_datetime(df[fullName('tpep_pickup_datetime')])
df[fullName('tpep_dropoff_datetime')] = pd.to_datetime(df[fullName('tpep_dropoff_datetime')])
df[fullName('duration')] = (df[fullName("tpep_dropoff_datetime")] - df[fullName("tpep_pickup_datetime")]).dt.total_seconds()

# feature engineering
# Feature engineering is the process of transforming raw data into inputs for a machine learning algorithm
df[fullName("weekday")] = (df[fullName('tpep_pickup_datetime')].dt.dayofweek < 5).astype(float)
df[fullName("hour")] = df[fullName('tpep_pickup_datetime')].dt.hour
df[fullName("work")] = (df[fullName('weekday')] == 1) & (df[fullName("hour")] >= 8) & (df[fullName("hour")] < 18)
df[fullName("month")] = df[fullName('tpep_pickup_datetime')].dt.month
# convert month to a categorical feature using one-hot encoding
df = pd.get_dummies(df, columns=[fullName("month")])

# Filter dataset to rides under 3 hours and under 150 miles to remove outliers
df = df[df[fullName('duration')] > 20]
df = df[df[fullName('duration')] < 10800]
df = df[df[fullName('trip_distance')] > 0]
df = df[df[fullName('trip_distance')] < 150]

# drop null rows
df = df.dropna(how='any',axis=0)

# select columns to be used as features
cols = [fullName('work'), fullName('startstationlatitude'), fullName('startstationlongitude'), fullName('endstationlatitude'), fullName('endstationlongitude'), fullName('trip_distance'), fullName('weekday'), fullName('hour')]
cols.extend([fullName('month_' + str(x)) for x in range(1, 7)])
cols.append(fullName('duration'))
dataset = df[cols]

# separate data into features (the taxi ride properties) and label (duration) using .iloc
X = dataset.iloc[:, 0:(len(cols) - 1)].values
y = dataset.iloc[:, (len(cols) - 1)].values
X = X.copy()
y = y.copy()
print (X.shape)
print (y.shape)
del dataset
del df

print("Done cleaning data")


print("Training and testing...")

# As we have one dataset, the data is split into a training data set and a test data set. The ideal split is 80:20. 
# 80% of the data is used to train the model and 20% is used for testing the model.
print("Load the sklearn module to split the dataset into a training data set and a test data set.")
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


print("Build the model and fit the model on the training data. This will take a few minutes...")

#Define the ML algorithm (that is the learning algorithm to use). We use here the XGBRegressor class of the xgboost package to build the model
xgbr = xgb.XGBRegressor(objective ='reg:squarederror', 
                        colsample_bytree = 1,
                        subsample = 1,
                        learning_rate = 0.15,
                        booster = "gbtree",
                        max_depth = 1,
                        eta = 0.5,
                        eval_metric = "rmse",) 

print("num train elements: " + str(len(X_train)))

# then train (fit) the model on training data set comprised of inputs (features) and outputs (label)
# we provide our defined ML algorithm with data to learn from
print("Train start time: ", datetime.datetime.now())
xgbr.fit(X_train, y_train)
#after training the model, check the model training score. The closer towards 1, the better the fit.
score = xgbr.score(X_train, y_train)  
print("Model training score. The closer towards 1, the better the fit: ", score)

print("Train end time: ", datetime.datetime.now())
print("Test the model by making predictions on the test data set where only the features are provided")
y_pred = xgbr.predict(X_test)
y_pred = y_pred.clip(min=0)

print("Evaluating the model accuracy by comparing the test predictions with the test labels. RMSE is used as evaluation metric.")
# evaluating the model accuracy
from sklearn import metrics
from sklearn.metrics import mean_squared_log_error
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('Root Mean Squared Log Error:', np.sqrt(mean_squared_log_error( y_test, y_pred)))
print()
print("Note that for RMSE the lower that value is, the better the fit")

#after we have trained the model, save it in a pickle file in the tenant user's working directory for later use in production deployment engine
#the model will be loaded back from the pickle file using the python scoring script in the deployment engine to make predictions on new data.
print("Saving model in a pickle file as " + ProjectRepo(studentRepoModel) + '/' + "XGB.pickle.dat")
pickle.dump(xgbr, open( ProjectRepo(studentRepoModel) + '/' + "XGB.pickle.dat", "wb"))

# Finish time
print("End time: ", datetime.datetime.now())

### **5- Monitor the training job**

#### Copy the unique log _History URL_ above and paste it into the cell below to monitor your training job (run the cell code below regularly until the job is marked as "**finished**")

>**Note:** Depending on the number of concurrent jobs submitted to the training cluster environment (i.e: multiple participants run the workshop concurrently), the model training may take several minutes to complete.

In [ ]:
##%logs --url your_http_url_here
%logs --url your_http_url_here

### **6- Model registry and deployment**

##### After the model development and training, you are now ready to deploy your trained model as prediction service and start serving queries.

<font color="red">**Now, let's go back to your JupyterHub account session to continue the hands-on from Lab 4 for model registry and model deployment:**</font>
<font color="blue">**4-WKSHP-MLOps-K8s-Register-Model-Deployment.ipynb**.</font>

**======================================================================================**